In [1]:
import pandas as pd

### Create inverse rates

In [127]:
df = pd.read_csv('testData.csv')
# append copy of dataframe to create inverse rates
df = (df
        .append(df)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
     )

In [128]:
for index in df[103:].index:
    # inverse rate
    df.loc[index, 'Rate'] = float(1/df.loc[index, 'Rate'])
    
    # create inverse numerator and denumerator names
    df.loc[index, 'Numerator1'] = df.loc[index, 'Denumerator']
    df.loc[index, 'Denumerator1'] = df.loc[index, 'Numerator']
    
    # reassign numerator and denumerator names
    df.loc[index, 'Numerator'] = df.loc[index, 'Numerator1']
    df.loc[index, 'Denumerator'] = df.loc[index, 'Denumerator1']

df = df.drop(columns=['index', 'Numerator1', 'Denumerator1'])
display(df.head(3))
df.shape

,Currency,Pair,Volume,Rate,Numerator,Denumerator
0,Paxos Standard Token,PAX/USDT,2.508283e+09,1.026300,PAX,USDT
1,USD Coin,USDC/USDT,1.513212e+09,1.026290,USDC,USDT
2,BitMax Token,BTMX/USDT,1.500912e+07,0.112087,BTMX,USDT


(206, 6)

### Create possible pairs in triangular arbitrage

In [86]:
pairList = []
# for every denumerator that has BTC as numerator
for i in list(df.Denumerator[df['Numerator'] == 'BTC']):
    # for every denumerator that has i as numerator
    for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != 'BTC') & (df['Denumerator'] != 'BTC')]))):
        if j in list(df.Denumerator[df['Numerator'] == 'BTC']):
            pairList.append([i,j])
len(pairList)

120

### Data preparation function

In [2]:
def prepareData(df):
    # INVERSE RATES CREATION
    df = (df
        .append(df)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
     )
    for index in df[103:].index:
        # inverse rate
        df.loc[index, 'Rate'] = float(1/df.loc[index, 'Rate'])

        # create inverse numerator and denumerator names
        df.loc[index, 'Numerator1'] = df.loc[index, 'Denumerator']
        df.loc[index, 'Denumerator1'] = df.loc[index, 'Numerator']

        # reassign numerator and denumerator names
        df.loc[index, 'Numerator'] = df.loc[index, 'Numerator1']
        df.loc[index, 'Denumerator'] = df.loc[index, 'Denumerator1']

    df = df.drop(columns=['index', 'Numerator1', 'Denumerator1'])
    
    # POSSIBLE PAIRS CREATION
    pairList = []
    # for every denumerator that has BTC as numerator
    for i in list(df.Denumerator[df['Numerator'] == 'BTC']):
        # for every denumerator that has i as numerator
        for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != 'BTC') & (df['Denumerator'] != 'BTC')]))):
            if j in list(df.Denumerator[df['Numerator'] == 'BTC']):
                pairList.append([i,j])
    
    return df, pairList

### Real data arbitrage

In [3]:
def triangularArbitrage(inputSet, firstTransaction, secondTransaction):
    fee = 0.0004
    # first transaction
    output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
    output1 = output1 - fee*output1
    
    # second transaction
    output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
    output2 = output2 - fee*output2
    
    # third transaction
    output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
    output2 = output2 - fee*output2
    
#     if float(output3 - inputSet[0]) > 0:
#         print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])
    
    return float(output3 - inputSet[0])

# inputSet = (1, 'BTC')
# %time triangularArbitrage(inputSet, 'LTC', 'ETH')

In [5]:
%%time
# read data
df = pd.read_csv('testData.csv')
# transform dataframe
df, pairList = prepareData(df)
suma = 0
n = 0
btcQuantity = 1
for i in pairList:
    add = triangularArbitrage((btcQuantity, 'BTC'), i[0], i[1])
    if add > 0:
        suma += add
        btcQuantity += add
        n += 1
print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} triangular transactions using cummulated BTC asset')

Obtained: 5781.32 PLN by 44 triangular transactions using cummulated BTC asset
Wall time: 812 ms
